In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import scipy as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import ttest_ind
from scipy import stats
from scipy.stats import boxcox
from scipy.stats import pearsonr
import scipy.stats as st

In [ ]:
df_demo = pd.read_csv('../Data/clean/demo_clean.csv')
df_digital = pd.read_csv('../Data/clean/digital_footprints.csv')
df_experiment = pd.read_csv('../Data/clean/clean_final_experiment.csv')

In [ ]:
df_digital.head()

In [ ]:
df_demo.head()

In [ ]:
df_experiment.head()

In [ ]:
# Using concat to combine df_digital, df_demo, and df_experiment

df = pd.concat([df_digital, df_demo, df_experiment], axis=1, join='inner')

df.head()

In [ ]:


# Convert the 'date_time' column to datetime 
df['date_time'] = pd.to_datetime(df['date_time'])

# Sort the DataFrame by date_time to ensure chronological order
df = df.sort_values('date_time')

# Calculate the time differences between consecutive steps
df['duration_between_steps'] = df['date_time'].diff()


df = df.sort_values(['process_step', 'date_time'])
df['duration_between_steps'] = df.groupby('process_step')['date_time'].diff()

# Option 1: Specify that day comes first
df['date_time'] = pd.to_datetime(df['date_time'], dayfirst=True)

print(df)

In [ ]:
import pandas as pd


df = pd.DataFrame(df)

df['date_time'] = pd.to_datetime(df['date_time'])
df = df.sort_values(['process_step', 'date_time'])
df['duration_between_steps'] = df.groupby('process_step')['date_time'].diff()

print(df)


In [ ]:
import pandas as pd


# Create DataFrame
df = pd.DataFrame(df)

# Convert to datetime
df['date_time'] = pd.to_datetime(df['date_time'])

# Sort by process and time
df = df.sort_values(['process_step', 'date_time'])

# Calculate duration between steps within each process
df['duration_between_steps'] = df.groupby('process_step')['date_time'].diff()

# Optional: Convert duration to minutes (or hours)
df['duration_minutes'] = df['duration_between_steps'].dt.total_seconds() / 60

print(df)


In [ ]:
df_filtered = df.dropna(subset=['duration_between_steps'])
df_filtered

In [ ]:
total_duration_minutes = df.groupby('process_step')['duration_minutes'].sum()
print(total_duration_minutes)


In [ ]:

# Drop NaT durations for plotting
plot_df = df.dropna(subset=['duration_minutes'])

# Bar plot of durations
plt.figure(figsize=(8, 5))
for label, group in plot_df.groupby('process_step'):
    plt.plot(group['date_time'], group['duration_minutes'], marker='o', label=f'Process {label}')

plt.xlabel('Time')
plt.ylabel('Duration between steps (minutes)')
plt.title('Step Durations by Process')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Add a cumulative duration column
df['cumulative_minutes'] = df.groupby('process_step')['duration_minutes'].cumsum()

print(df)


In [ ]:
df['cumulative_minutes'] = df.groupby('process_step')['duration_minutes'].cumsum()


plt.figure(figsize=(10, 6))

# Plot each process line
for name, group in df.groupby('process_step'):
    plt.plot(group['date_time'], group['cumulative_minutes'], marker='o', label=f'Process {name}')

plt.title('Cumulative Duration Curve per Process')
plt.xlabel('Time')
plt.ylabel('Cumulative Duration (Minutes)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
mean_duration_minutes = df.groupby('process_step')['duration_minutes'].mean()
print(mean_duration_minutes)

In [ ]:
mean_duration = df[df['duration_minutes'] > 0].groupby('process_step')['duration_minutes'].mean()
mean_duration

In [ ]:
plt.figure(figsize=(6, 4))
mean_duration.plot(kind='bar', color='skyblue', edgecolor='black')

# Customize plot
plt.title('Mean Duration Between Steps per Process')
plt.xlabel('Process Step')
plt.ylabel('Mean Duration (Minutes)')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

In [ ]:
mean_duration_sorted = mean_duration.sort_values(ascending=False)  # Set to True for ascending
plt.figure(figsize=(6, 4))
mean_duration_sorted.plot(kind='bar', color='skyblue', edgecolor='black')

# Customize plot
plt.title('Mean Duration Between Steps per Process (Sorted)')
plt.xlabel('Process Step')
plt.ylabel('Mean Duration (Minutes)')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

In [ ]:
df.columns

In [ ]:
# Calculate completion rate per day using your existing columns
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Extract date from date_time column
df['date'] = pd.to_datetime(df['date_time']).dt.date

# Calculate total cumulative minutes per day
# This gets the maximum cumulative minutes for each day
daily_totals = df.groupby('date')['cumulative_minutes'].max().reset_index()
daily_totals.rename(columns={'cumulative_minutes': 'completion_rate_per_day'}, inplace=True)

# Create histogram
sns.histplot(daily_totals['completion_rate_per_day'], 
             bins=20, kde=True, color='skyblue', edgecolor='black')

plt.title('Histogram with Density Plot: Completion Rate per Day')
plt.xlabel('Completion Rate per Day (Cumulative Minutes)')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate completion rate per day using process_step and duration_between_steps
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Extract date from date_time column
df['date'] = pd.to_datetime(df['date_time']).dt.date

# Calculate completion metrics
# Group by date and find the average duration between steps
completion_rate_day = df.groupby('date')['duration_between_steps'].mean().reset_index()
completion_rate_day.rename(columns={'duration_between_steps': 'completion_rate_per_day'}, inplace=True)

# Alternative: Count number of process steps completed per day


step_counts = df.groupby(['date', 'process_step']).size().reset_index(name='count')
completion_rate_day = step_counts.groupby('date')['count'].sum().reset_index()
completion_rate_day.rename(columns={'count': 'completion_rate_per_day'}, inplace=True)

# Create histogram
sns.histplot(completion_rate_day['completion_rate_per_day'], 
             bins=20, kde=True, color='skyblue', edgecolor='black')

plt.title('Histogram with Density Plot: Completion Rate per Day')
plt.xlabel('Avg Duration Between Steps per Day (minutes)')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()